In [ ]:
import os
import sys
import requests as req
import json

from datetime import datetime

import pandas as pd
import matplotlib.pyplot as plt

new_path = os.getcwd().replace('/notebooks', '')
print(f'Adding {new_path} to sys.path')

if new_path not in sys.path:
    sys.path.append(new_path)

# Local imports
from curvemetrics.src.classes.datahandler import DataHandler
from curvemetrics.src.classes.datafetcher import DataFetcher
from curvemetrics.src.classes.metricsprocessor import MetricsProcessor

In [ ]:
plt.rcParams["font.family"] = "serif"
plt.rcParams.update({'font.size': 10})

In [ ]:
datahandler = DataHandler()
token_metadata = datahandler.get_token_metadata()
pool_metadata = datahandler.get_pool_metadata()
for p in pool_metadata:
    pool_metadata[p]['inputTokens'] = json.loads(pool_metadata[p]['inputTokens'])

pool = "0xbebc44782c7db0a1a60cb6fe97d0b483032ff1c7" # 3pool

start_ts, start_block = DataFetcher.get_block(datetime.fromisoformat("2023-03-01"))
end_ts, end_block = DataFetcher.get_block(datetime.fromisoformat("2023-03-15"))

In [ ]:
async def get_snapshots():
    datafetcher = DataFetcher(token_metadata=token_metadata)
    try:
        snapshots = datafetcher.get_snapshots(start_ts, end_ts, pool)
    except Exception as e:
        print(f'excepted: {e}')
        return 
    finally:
        await datafetcher.close()
    return snapshots
    
snapshots = await get_snapshots()

In [ ]:
df = pd.DataFrame.from_dict([x for y in snapshots for x in y])
for col in ['A', 'offPegFeeMultiplier', 'timestamp', 'virtualPrice', 'lastPricesTimestamp', 'block_gte', 'block_lt']:
    df[col] = df[col].astype(int)
for col in ['adminFee', 'fee', 'lpPriceUSD', 'tvl', 'totalDailyFeesUSD', 'lpFeesUSD']:
    df[col] = df[col].astype(float)
for col in ['normalizedReserves', 'reserves']:
    df[col] = df[col].apply(lambda x: [int(y) for y in x])
for col in ['reservesUSD']:
    df[col] = df[col].apply(lambda x: [float(y) for y in x])
df = df.sort_values(by='timestamp')
df.index = df['timestamp'].apply(datetime.fromtimestamp)
df.head()